In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pandas.plotting import autocorrelation_plot
from statsmodels.graphics.tsaplots import plot_pacf

from statsmodels.tsa.holtwinters import SimpleExpSmoothing, ExponentialSmoothing
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.discrete.discrete_model import NegativeBinomialP,NegativeBinomial,Poisson
from statsmodels.graphics.boxplots import violinplot
from scipy.stats import boxcox
from scipy.special import inv_boxcox
import pywt
from statsmodels.robust import mad
from statsmodels.discrete.count_model import ZeroInflatedNegativeBinomialP
from statsmodels.discrete.count_model import ZeroInflatedPoisson
from statsmodels.tsa.api import VAR
from statsmodels.tsa.api import VARMAX
from statsmodels.tsa.statespace.dynamic_factor import DynamicFactor
from statsmodels.tsa.api import Holt
from statsmodels.tsa.regime_switching.markov_regression import MarkovRegression

import seaborn as sns
from scipy.stats import nbinom
from statistics import mean,stdev

from sklearn.ensemble import RandomForestRegressor
from scipy import stats  

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import time

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
# What needs to be forecasted; daily sales for the next 28 days per item per store.
sales_data = pd.read_csv("/kaggle/input/m5-forecasting-accuracy/sales_train_validation.csv")
price_data = pd.read_csv("/kaggle/input/m5-forecasting-accuracy/sell_prices.csv")
calender_data = pd.read_csv("/kaggle/input/m5-forecasting-accuracy/calendar.csv")
submission_data = pd.read_csv("/kaggle/input/m5-forecasting-accuracy/sample_submission.csv")

In [ ]:
features = calender_data[{'wday','month','year','snap_CA','snap_TX','snap_WI','event_type_1','event_type_2'}]
features = features[0:len(features)]
features = features.fillna(value=0)
features = features.replace("Religious",1)
features = features.replace("National",2)
features = features.replace("Sporting",3)
features = features.replace("Cultural",4)

In [ ]:
class Data:
    features = calender_data[{'wday','month','year','snap_CA','snap_TX','snap_WI'}]
    
    #features = calender_data[{'wday','month','year','snap_CA','snap_TX','snap_WI','event_type_1','event_type_2'}]
    #features = features[0:len(features)]
    #features = features.fillna(value=0)
    #features = features.replace("Religious",1)
    #features = features.replace("National",2)
    #features = features.replace("Sporting",3)
    #features = features.replace("Cultural",4)
    
    def __init__(self, item, store, dept, t1, t2):
        self.item = item
        self.store = store
        self.dept = dept
        self.t1 = t1
        self.t2 = t2
    
    
    # Custom data extraction function meant for looking at single series
    def gen_data(self):
        t1=self.t1; t2=self.t2
        dept_sum = sales_data.groupby(['dept_id']).sum().T.reset_index(drop = True)
        date_col = [col for col in sales_data if col.startswith('d_')]
        sales_sum = pd.DataFrame(sales_data[date_col].sum(axis =0),columns = ["sales"]);
        item = self.item
        store = self.store
        dept = self.dept    
        
        if dept == True:
            data_store_foods = sales_data.groupby(['dept_id','store_id']).sum().T.reset_index(drop = True)
            exp_data_total = data_store_foods[item,store]

        if dept == False:
            data_store_foods = sales_data.groupby(['item_id','store_id']).sum().T.reset_index(drop = True)
            exp_data_total = data_store_foods[item,store]
            
        exp_data_train = exp_data_total[0:t1]

        self.total = data_store_foods[item,store]
        self.train = exp_data_train.values
        if t2 < len(exp_data_total):
            self.val = exp_data_total[t1:t2]
        else:
            self.val = "ERROR: Test set required exceeds data available."
        return 0
    
    def preprocess(self, data):
        return denoise_signal(data)
    
    def post_process(self,y, lmbda):    
        return inv_boxcox(y,lmbda) # = y,fitted_lambda 
    
    def forecast_ARIMA(self, data):
        t1=len(data); t2=len(data)+self.t2-self.t1
        model = SARIMAX(data,order=(7,2,7),enforce_stationarity=False)
        model_fit = model.fit(full_output=0,disp=0)

        x = np.linspace(5,len(data),len(data))
        
        self.ARIMA_predict_train = model_fit.predict(0,len(data),typ='levels')
        self.ARIMA_predict_forecast = model_fit.predict(t1,t2-1,typ='levels')
        return 0
    
        
    def forecast_NEGBIN(self, datal):
        t1=self.t1; t2=self.t2
        data = datal
        exog = self.features
        model = NegativeBinomialP(data, exog[t1-len(data):t1])
        model_fit = model.fit(method='nm',full_output=0,disp=0)

        self.NEGBIN_predict_train = model_fit.predict(exog)
        self.NEGBIN_predict_forecast = model_fit.predict(exog[t1:t2])
        return 0
    
    def forecast_MARKOV(self, datal):
        t1=self.t1; t2=self.t2
        data = datal
        exog = self.features
        k_regimes=3
        model = MarkovRegression(data,k_regimes,trend="ct")
        model_fit = model.fit(method='nm',full_output=0,disp=0)
        x = np.linspace(0,len(data),len(data))
        self.VECTOR_predict_forecast = model_fit.predict(100-56)
        return 0
    
    def forecast_SES(self, datal):
        t1=self.t1; t2=self.t2
        data = datal
        exog = self.features
        k_regimes=3
        model = ExponentialSmoothing(data)
        model_fit = model.fit()
        x = np.linspace(0,len(data),len(data))
        self.SES_predict_forecast = model_fit.predict(100-56)
        return 0
    
    def forecast_POISSON(self, datal):
        t1=self.t1; t2=self.t2
        data = datal[0:len(datal)]
        exog = self.features
        model = Poisson(data, exog[t1-len(datal):t1])
        model_fit = model.fit(full_output=0,disp=0)

        self.POISSON_predict_train = model_fit.predict(exog)
        self.POISSON_predict_forecast = model_fit.predict(exog[t1:t2])

        
        return 0
    

**Forecasting stuff**

In [ ]:
def make_submission():
    store_data = sales_data["store_id"]
    item_data = sales_data["item_id"]
    anal_end = len(sales_data)
    val_days = 56
    submission_out = np.zeros((anal_end,56))
    submission_val = np.zeros((anal_end,28))
    submission_eva = np.zeros((anal_end,28))

    start_time = time.time()

    train_data = sales_data.drop(columns=['id','cat_id','item_id','dept_id','store_id','state_id']).values
    eri = 0
    erih = 0
    for i in range(anal_end):
        dat = train_data[i]
        dat0 = Data(item_data[i], store_data[i], False, 1913, 1969) # create data object for every series, extrapolate from 1913 to 1969
        preproc = dat
        try:
            dat0.forecast_POISSON(preproc[-101:-1]); # Run the method to train and forecast the single series
            foreout = dat0.POISSON_predict_forecast
            
            comp = mean(map(float, preproc[-21:-1]))+4*stdev(map(float,preproc[-21:-1]))
            if np.any(foreout > comp) == True:
                foreout = mean(preproc[-1969-1913:-1])
                erih = erih+1
        except: # for when the method fails to converge
            foreout = mean(preproc[-1969-1913:-1])
            eri = eri+1
        
        if np.isnan(np.min(foreout)) == True: # check for numbers
            foreout = mean(preproc[-1969-1913:-1])
            
        submission_out[i] = np.round(foreout,3)
        

    submission_out[submission_out<0]=0
    
    print("Execution time: ",time.time()-start_time)
    
    # split submission into a validation (public scoreboard) and evaluation (private scoreboard) sets
    submission_val = submission_out[:,0:28]
    submission_eva = submission_out[:,28:56]

    ids_tot = submission_data['id'].values
    ids_val = pd.DataFrame(data=ids_tot[0:30489], columns=['id'])
    ids_eva = pd.DataFrame(data=ids_tot[30490:], columns=['id'])

    submission_val = pd.DataFrame(data=submission_val, columns=[f'F{i+1}' for i in range(28)])
    submission_val = pd.concat([ids_val,submission_val],axis=1)

    submission_eva = pd.DataFrame(data=submission_eva, columns=[f'F{i+1}' for i in range(28)])
    submission_eva = pd.concat([ids_eva,submission_eva],axis=1)

    submission = pd.concat([submission_val,submission_eva]).reset_index(drop=True)
    submission['id'] = submission_data['id']
    print("Number of crashes: ", eri)
    print("Number of values exceeding mu+4*sigma: ", erih)
    submission.to_csv('submission.csv', index=False)

In [ ]:
make_submission()